<a href="https://colab.research.google.com/github/HaliullinAnton/DIploma/blob/main/Data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def create_table(table_name, columns_name, columns_quant, file_loc, head_pass):
    conn = sqlite3.connect(r'/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute('''DROP TABLE IF EXISTS %s''' % (table_name))
    cur.execute('''CREATE TABLE %s (%s)'''%(table_name, columns_name) )
    with open(r'%s' % (file_loc), 'r') as fin_1:
        dr = csv.reader(fin_1)
        if head_pass == True:
            next(dr, None)  # пропускаем заголовок
        else:
            pass
        dataset = [(i) for i in dr]
    for count, i in enumerate(dataset):
        dataset[count] = tuple(i)
    dataset = tuple(dataset)

    columns_count = ""
    for i in range(columns_quant):
        if i < (columns_quant - 1):
            columns_count += '?, '
        else: columns_count += '?'
    cur.executemany("INSERT INTO %s VALUES (%s);" % (table_name, columns_count), dataset)
    conn.commit()
    conn.close()
    '''
    Функция загружает csv файл и загружает данные из него в БД
    Первый аргумент - название таблицы, второй - название столбцов в таблице,
     третий - путь к csv файлу, четвертый - булево значение, где True - 
     пропустить заголовок, False - оставить заголовок
    '''

In [ ]:
def func_cur(sql_exp):
    conn = sqlite3.connect('/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute(''' %s;''' %(sql_exp))
    result = cur.fetchall()
    return pd.DataFrame(result)
    '''
    Функция создания курора. В аргумент функции передается полный SQL-запрос.
    '''
 
 
def select_all(table_name):
    conn = sqlite3.connect('/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute(''' SELECT * FROM %s;''' %(table_name))
    result = cur.fetchall()
    return pd.DataFrame(result)
    '''
    Функция создания курсора, и вывода всех столбцов из заданной таблицы.
    В аргумент передается только название таблицы
    '''

def show_db():
    conn = sqlite3.connect('/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute(''' SELECT name FROM sqlite_master WHERE type = 'table';
    ''')
    result = cur.fetchall()
    return result
    '''
    Функция выводит названия таблиц в базе данных
    '''

In [ ]:
#Преобразуем csv файл для корректного чтения
train_data = pd.read_csv(r'/content/drive/MyDrive/Diploma/Data/Dict/train.csv', sep=';')
pd.DataFrame(train_data).to_csv(r'/content/drive/MyDrive/Diploma/Data/Dict/train_sep.csv', index=False)


In [ ]:
#Загружаем таблицу train
create_table('train', 'train_id, car_id', 2, '/content/drive/MyDrive/Diploma/Data/Dict/train_sep.csv', True)

In [ ]:
# Загружаем таблицу users
create_table('users', 'user_id', 1, '/content/drive/MyDrive/Diploma/Data/Dict/users.csv', True)

In [ ]:
# Загружаем таблицу stations
import json
with open('/content/drive/MyDrive/Diploma/Data/Dict/station.json','r') as f:
    j = json.load(f)
l = list(j.values())
df = pd.DataFrame(data=l)
df_t = df.transpose() # Транспонируем матрицу, т.к. таблица горизонтальная
df_t = df_t.fillna('No_Data') # Заполняем пустые значения
pd.DataFrame(df_t.to_csv(r'/content/drive/MyDrive/Diploma/Data/Dict/station.csv', index=False)) # Создаем csv файл
df_t
create_table('stations', 'line_id, station_name, line_name, district, district_1, acts, station_id', 7, '/content/drive/MyDrive/Diploma/Data/Dict/station.csv', True)

In [ ]:
# Загражаем таблицы hw_session_1 - hw_session_17
for i in range(1, 18):
  create_table('hw_session_%s'%(i), 'user_id, stst_id, stopst_id, start_ts, end_ts, start_dt, end_dt', 7, '/content/drive/MyDrive/Diploma/Data/Table/hw_session_%s.csv'%(i), True)


In [ ]:
# Объединяем таблицы hw_sessions

hw_session = func_cur('''SELECT * FROM hw_session_1 UNION ALL SELECT * FROM 
hw_session_2 UNION ALL SELECT * FROM hw_session_3 UNION ALL SELECT * FROM
hw_session_4 UNION ALL SELECT * FROM hw_session_5 UNION ALL SELECT * FROM 
hw_session_6 UNION ALL SELECT * FROM hw_session_7 UNION ALL SELECT * FROM 
hw_session_8 UNION ALL SELECT * FROM hw_session_9 UNION ALL SELECT * FROM
hw_session_10 UNION ALL SELECT * FROM  hw_session_11 UNION ALL SELECT * FROM
hw_session_12 UNION ALL SELECT * FROM hw_session_13 UNION ALL SELECT * FROM
hw_session_14 UNION ALL SELECT * FROM hw_session_15 UNION ALL SELECT * FROM
hw_session_16 UNION ALL SELECT * FROM hw_session_17''')

In [ ]:
# Создаем таблицу hw_session
hw_session = list(hw_session.itertuples(index=False, name=None))
hw_session = tuple(hw_session)

conn = sqlite3.connect(r'/content/drive/MyDrive/Diploma/Data/New_db.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS hw_session')
cur.execute('CREATE TABLE hw_session (user_id, stst_id, stopst_id, start_ts, end_ts, start_dt, end_dt)')

cur.executemany("INSERT INTO hw_session VALUES (?, ?, ?, ?, ?, ?, ?);", hw_session)
conn.commit()
conn.close()


In [ ]:
# Удаляем лишние таблицы
for i in range(1,18):
    conn = sqlite3.connect(r'/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS hw_session_%s'%(i))
    conn.commit
    conn.close
  

In [ ]:
# Загражаем таблицы train_trek_1 - train_trek_17
for i in range(1, 18):
  create_table('train_trek_%s'%(i), 'line_id, train_id, station_id, timestamp, dt', 5, '/content/drive/MyDrive/Diploma/Data/Table/train_trek%s.csv'%(i), True)

In [ ]:
# Объединяем таблицы train_trek

train_trek = func_cur('''SELECT * FROM train_trek_1 UNION ALL SELECT * FROM 
train_trek_2 UNION ALL SELECT * FROM train_trek_3 UNION ALL SELECT * FROM
train_trek_4 UNION ALL SELECT * FROM train_trek_5 UNION ALL SELECT * FROM 
train_trek_6 UNION ALL SELECT * FROM train_trek_7 UNION ALL SELECT * FROM 
train_trek_8 UNION ALL SELECT * FROM train_trek_9 UNION ALL SELECT * FROM
train_trek_10 UNION ALL SELECT * FROM  train_trek_11 UNION ALL SELECT * FROM
train_trek_12 UNION ALL SELECT * FROM train_trek_13 UNION ALL SELECT * FROM
train_trek_14 UNION ALL SELECT * FROM train_trek_15 UNION ALL SELECT * FROM
train_trek_16 UNION ALL SELECT * FROM train_trek_17''')

In [ ]:
# Создаем таблицу hw_session
train_trek = list(train_trek.itertuples(index=False, name=None))
train_trek = tuple(train_trek)

conn = sqlite3.connect(r'/content/drive/MyDrive/Diploma/Data/New_db.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS train_trek')
cur.execute('CREATE TABLE train_trek (line_id, train_id, station_id, timestamp, dt)')

cur.executemany("INSERT INTO train_trek VALUES (?, ?, ?, ?, ?);", train_trek)
conn.commit()
conn.close()

In [ ]:
# Удаляем лишние таблицы
for i in range(1,18):
    conn = sqlite3.connect(r'/content/drive/MyDrive/Diploma/Data/New_db.db')
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS train_trek_%s'%(i))
    conn.commit
    conn.close

In [ ]:
#  Создаем таблицу points 
create_table('points', 'id, user_id, start_ts, start_dt', 4, '/content/drive/MyDrive/Diploma/Data/Table/points.csv', True)

In [ ]:
# Создаем таблицу town_points

town_points = pd.read_excel(r'/content/drive/MyDrive/Diploma/Data/Dict/town_points.xlsx') # Загружаем файл xlsx
town_points = town_points.fillna('No_data')  # Заполняем пустые значения
pd.DataFrame(town_points).to_csv(r'/content/drive/MyDrive/Diploma/Data/Dict/town_points.csv', index=False) # Создаем файл csv

town_points

create_table('town_points', 'location_id, type_location, name_location, adress, metro', 5, '/content/drive/MyDrive/Diploma/Data/Dict/town_points.csv', True)


In [ ]:
show_db()

[('infected',),
 ('train',),
 ('users',),
 ('stations',),
 ('hw_session',),
 ('train_trek',),
 ('points',),
 ('town_points',)]